In [1]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
from pyproj import Proj, CRS, transform
import matplotlib
import sys
sys.path.append('/home/ZhiLi/CRESTHH')
from cresthh.utils import flowAreaCalc as flow_area
from cresthh.utils import processSWW
from cresthh.anuga import SWW_plotter
from cresthh.anuga.utilities import sww_merge
from cresthh.anuga.shallow_water.forcing import Cross_section
from cresthh import metrics as met
%matplotlib inline
import geopandas as gpd
# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')

In [2]:
from pyproj import Proj, CRS, transform
myProj = Proj("+proj=utm +zone=15, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

In [3]:
coupled= SWW_plotter('../calibration/coupled_opt.sww', start_time=pd.to_datetime('20170825120000'))

Figure files for each frame will be stored in _plot


In [4]:
noncoupled= SWW_plotter('../calibration/noncoupled_opt.sww', start_time=pd.to_datetime('20170825120000'))

Figure files for each frame will be stored in _plot


In [37]:
import mercantile

In [38]:
delta=0.13
tl= [30.04+delta, -95.6-delta]
br= [29.81-delta, -95.17+delta]
z=13

# lat_lng = [40.7231212,-74.0173587]
# delta = 0.13
# tl = [lat_lng[0]+delta, lat_lng[1]-delta]
# br = [lat_lng[0]-delta, lat_lng[1]+delta]
# z = 13 # Set the resolution

tl_tiles = mercantile.tile(tl[1],tl[0],z)
br_tiles = mercantile.tile(br[1],br[0],z)

x_tile_range = [tl_tiles.x,br_tiles.x];print(x_tile_range)
y_tile_range = [tl_tiles.y,br_tiles.y];print(y_tile_range)

[1917, 1933]
[3375, 3388]


In [46]:
myProj(br[1],br[0])

(302597.13350357197, 3285068.1805067253)

In [47]:
myProj(tl[1], tl[0])

(237098.16787431354, 3340772.208959892)

In [39]:
import requests
import shutil
for i,x in enumerate(range(x_tile_range[0],x_tile_range[1]+1)):
    for j,y in enumerate(range(y_tile_range[0],y_tile_range[1]+1)):
        print(x,y)
        r = requests.get('https://api.mapbox.com/v4/mapbox.terrain-rgb/'+str(z)+'/'+str(x)+'/'+str(y)+'@2x.pngraw?access_token=pk.eyJ1Ijoia2FwYXN0b3IiLCJhIjoiY2p3eTg3eWJoMG1jZjQ4bzZmcGg5c3F3cSJ9.vhyCyD9xDDGP9EQnhB9xtA', stream=True)
        if r.status_code == 200:
            with open('./elevation_images/' + str(i) + '.' + str(j) + '.png', 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)  
        
        r = requests.get('https://api.mapbox.com/v4/mapbox.satellite/'+str(z)+'/'+str(x)+'/'+str(y)+'@2x.png?access_token=pk.eyJ1Ijoia2FwYXN0b3IiLCJhIjoiY2p3eTg3eWJoMG1jZjQ4bzZmcGg5c3F3cSJ9.vhyCyD9xDDGP9EQnhB9xtA', stream=True)
        if r.status_code == 200:
            with open('./satellite_images/' + str(i) + '.' + str(j) + '.png', 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)

(1917, 3375)
(1917, 3376)
(1917, 3377)
(1917, 3378)
(1917, 3379)
(1917, 3380)
(1917, 3381)
(1917, 3382)
(1917, 3383)
(1917, 3384)
(1917, 3385)
(1917, 3386)
(1917, 3387)
(1917, 3388)
(1918, 3375)
(1918, 3376)
(1918, 3377)
(1918, 3378)
(1918, 3379)
(1918, 3380)
(1918, 3381)
(1918, 3382)
(1918, 3383)
(1918, 3384)
(1918, 3385)
(1918, 3386)
(1918, 3387)
(1918, 3388)
(1919, 3375)
(1919, 3376)
(1919, 3377)
(1919, 3378)
(1919, 3379)
(1919, 3380)
(1919, 3381)
(1919, 3382)
(1919, 3383)
(1919, 3384)
(1919, 3385)
(1919, 3386)
(1919, 3387)
(1919, 3388)
(1920, 3375)
(1920, 3376)
(1920, 3377)
(1920, 3378)
(1920, 3379)
(1920, 3380)
(1920, 3381)
(1920, 3382)
(1920, 3383)
(1920, 3384)
(1920, 3385)
(1920, 3386)
(1920, 3387)
(1920, 3388)
(1921, 3375)
(1921, 3376)
(1921, 3377)
(1921, 3378)
(1921, 3379)
(1921, 3380)
(1921, 3381)
(1921, 3382)
(1921, 3383)
(1921, 3384)
(1921, 3385)
(1921, 3386)
(1921, 3387)
(1921, 3388)
(1922, 3375)
(1922, 3376)
(1922, 3377)
(1922, 3378)
(1922, 3379)
(1922, 3380)
(1922, 3381)

In [ ]:
import PIL
import math
from os import listdir
from os.path import isfile, join
from glob import glob

for img_name in ['elevation','satellite']:
    image_files = [ f for f in glob('./'+img_name+'_images/*.png')]
    images = [PIL.Image.open(x) for x in image_files]

    edge_length_x = x_tile_range[1] - x_tile_range[0]
    edge_length_y = y_tile_range[1] - y_tile_range[0]
    edge_length_x = max(1,edge_length_x)
    edge_length_y = max(1,edge_length_y)
    width, height = images[0].size

    total_width = width*edge_length_x
    total_height = height*edge_length_y

    composite = PIL.Image.new('RGB', (total_width, total_height))

    anim_idx = 0
    y_offset = 0
    for i in range(0,edge_length_x):
        x_offset = 0
        for j in range(0,edge_length_y):
            tmp_img = PIL.Image.open('./'+img_name+'_images/' + str(i) + '.' + str(j) + '.png')
            composite.paste(tmp_img, (y_offset,x_offset))
            x_offset += width

            
        y_offset += height

    composite.save('./composite_images/'+img_name+'.png')

In [5]:
import matplotlib.image as mpimg
img = mpimg.imread('composite_images/satellite.png')

In [94]:
import matplotlib.tri as mtri
def interpolateField(splotter,field, xi, yi, frame):

    fea= field[frame]
        
#     interp_lin = mtri.CubicTriInterpolator(triang, z, kind='geom')
    triangles= splotter.volumes
    x= splotter.x
    y= splotter.y
    triang= mtri.Triangulation(x, y, triangles)
    interp_lin = mtri.CubicTriInterpolator(triang, fea, kind='geom')
    xi,yi= np.meshgrid(xi,yi)
    zi_lin = interp_lin(xi, yi)
    
    return xi, yi, zi_lin

In [6]:
dr= pd.date_range('20170825120000','20170831000000', freq='120S')

In [7]:
dr.get_loc('20170830154800')

3714

In [8]:
for i in range(3714, len(coupled.depth),1):
    if i%6==0:
        fig=plt.figure(figsize=(14,20))
        ax= fig.add_subplot(211)
        plt.imshow(np.flip(img,axis=0), extent=[235498.16787431354-coupled.xllcorner,300997.13350357197-coupled.xllcorner,
                                3286068.1805067253-coupled.yllcorner,3341772.208959892-coupled.yllcorner,],
                   origin='bottom left')
        # plt.tripcolor(coupled.triang,
        #               facecolors=coupled.elev,
        #               cmap='Greys_r')

        coupled.triang.set_mask(coupled.depth[i] < 0.5)
        plt.tripcolor(coupled.triang,
                      facecolors=coupled.depth[i],
                      cmap='GnBu',
                      vmin=0,
                      vmax=10,)
        plt.axis('off')
        plt.xlim([coupled.xc.min(),coupled.xc.max()+500])
        plt.ylim([coupled.yc.min()-1000,coupled.yc.max()+1000])
        plt.title('CREST-iMaP %s'%dr[i].strftime('%Y%m%d-%H:%M:%S'), fontsize=16, weight='bold')
        cbar= plt.colorbar(fraction=0.03, pad=0.04)
        cbar.set_label('depth (m)', fontsize=15);
        ax= fig.add_subplot(212)
        plt.imshow(np.flip(img,axis=0), extent=[235498.16787431354-coupled.xllcorner,300997.13350357197-coupled.xllcorner,
                                3286068.1805067253-coupled.yllcorner,3341772.208959892-coupled.yllcorner,],
                   origin='bottom left')
        # plt.tripcolor(coupled.triang,
        #               facecolors=coupled.elev,
        #               cmap='Greys_r')

        noncoupled.triang.set_mask(noncoupled.depth[i] < 0.5)
        plt.tripcolor(noncoupled.triang,
                      facecolors=noncoupled.depth[i],
                      cmap='GnBu',
                      vmin=0,
                      vmax=10,);
        plt.xlim([coupled.xc.min(),coupled.xc.max()+500])
        plt.ylim([coupled.yc.min()-1000,coupled.yc.max()+1000])        
        plt.axis('off')
        plt.title('hydraulic %s'%dr[i].strftime('%Y%m%d-%H:%M:%S'), fontsize=16, weight='bold')
        cbar= plt.colorbar(fraction=0.03, pad=0.04)
        cbar.set_label('depth (m)', fontsize=15);
        
        fig.savefig('gifs/%s.png'%dr[i].strftime('%Y%m%d-%H:%M:%S'))
        plt.tight_layout();
        plt.close();
        del fig
    else:
        pass

(-15259.180937280471,
 50239.784691977955,
 -15261.838871004526,
 40442.18958216207)

(3.0513804, 37790.921875)

(-997.5638937950134, 23882.302734375)

Text(0.5,1,'CREST-iMaP 20170830-15:48:00')

(3.0513804, 37790.921875)

...

Text(0.5,1,'CREST-iMaP 20170831-00:00:00')

(3.0513804, 37790.921875)

(-997.5638937950134, 23882.302734375)

(3.0513803958892822, 37790.921875, -997.5638937950134, 23882.302734375)

Text(0.5,1,'hydraulic 20170831-00:00:00')